# HCC mapping in Python
This first part of this library consists of 3 helper files that clean up the crosswalk mappings and extract the Condition Categories and Hierarchies found at **[CMS.gov](https://www.cms.gov/Medicare/Health-Plans/MedicareAdvtgSpecRateStats/Risk-Adjustors.html)**. As I have already generated the files for all CC versions between 2009-2017, these only need to be run if other versions or years are needed.
  
### format_crosswalks():
For every year between 2009 and 2017 and all CC versions, raw data was downloaded and placed in *Raw/year/version/*. The original crosswalk files are labeled similarly, like `F1209F1Y.txt`. These format of the raw files is as follows

    0031   2   
    00322  112 
    00323  37  
    00324  37  
    0064   112 
The first column is the icd code, the second column is the Condition Category, and sometimes there is an extra label of `'D'` following the condition code if the icd code has mutliple CC mappings. For years before 2016 and versions 12 and 21, instead of the `D` notation, additional mappings are found in files like `V12H70M.txt` and were manually added to the crosswalk list (through a logic loop), so the resulting crosswalks are still considered complete. The output of this script is a set of nicely formatted crosswalk csvs found in *Crosswalks/* specifying the year, CC version and icd version used in the mappings. The columns have headings and I also append the ICD version (9 or 10) as `version` and the `year` that the mapping is valid for.

In [1]:
import pandas as pd
df = pd.read_csv('Crosswalks/2009_v12_icd9.csv')
df.head()

,cc,icd,version,year
0,2,0031,9,2009
1,112,00322,9,2009
2,37,00323,9,2009
3,37,00324,9,2009
4,112,0064,9,2009


Years before 2009 and after 2017 can be added by downloading the data to the appropriate folder in *'Raw/* and then calling `format_crosswalks()` with the file in the appropriate `icd9_list` or `icd10_list` argument. For years before 2009, also be sure to update the logic that manually appends the additional CCs found in the `V12H870M.txt` files. Years after 2017 simply have `D` in the Raw mapping file rows to indicate additional mappings, and `format_crosswalks()` will read in the duplicates without issue in that case.

### extract_cc_table()
Extracts the Condition Codes and Condition Code labels for a given CC version. Raw data is found in *Raw/year/version* and the label files are named like `V12H70L1.txt`. This file only changes with version, so choosing a file from any year is appropriate. For v12, the raw format is of the form:

    ***********************************************************************;
    LABEL
    HCC1  ='HIV/AIDS                                                       '
    HCC2  ='Septicemia/Shock                                               ''
    
and for v21 and v22 this format changed slightly to:

    ***********************************************************************;
    LABEL
    HCC1 ="HIV/AIDS"
      HCC2 ="Septicemia, Sepsis, Systemic Inflammatory Response Syndrome/Shock"
Note, that several of the codes spanned mutiple lines in the original Raw data, and I manually changed those lines to span one line in the Raw files so the regex would work. The condition codes and labels are separated with regex (dependent upon the version) and are placed in a formatted csv found in *ConditionCategory/*, labeled by the version number with the suffix *_labels*.

In [2]:
# Labels file for v12
df_label = pd.read_csv('ConditionCategory/v12_labels.csv')
df_label.head()

,cc,label
0,1,HIV/AIDS ...
1,2,Septicemia/Shock ...
2,5,Opportunistic Infections ...
3,7,Metastatic Cancer and Acute Leukemia ...
4,8,"Lung, Upper Digestive Tract, and Other Severe ..."


### extract_rules()
Extracts the hierarchical rules for a given CC version. Raw data is found in *Raw/year/version* and the hierarchy files are named like `V12H70H.txt`. For v12, the raw files are of the form:

     %*imposing hierarchies;
     /* Infection 5 */ if hcc5   =1 then do i=112;hcc(i)=0;end;
     /* Neoplasm1 */   if hcc7   =1 then do i=8,9,10;hcc(i)=0;end;
     /* Neoplasm 2 */  if hcc8   =1 then do i=9,10;hcc(i)=0;end;
     /* Neoplasm 3 */  if hcc9   =1 then do i=10;hcc(i)=0;end;
     /* Diabetes 1 */  if hcc15  =1 then do i=16,17,18,19;hcc(i)=0;end;
     
and for v21 and v22, the format is different:

     %*imposing hierarchies;
     /*Neoplasm 1 */   %SET0(CC=8     , HIER=%STR(9 ,10 ,11 ,12 ));
     /*Neoplasm 2 */   %SET0(CC=9     , HIER=%STR(10 ,11 ,12 ));
     /*Neoplasm 3 */   %SET0(CC=10    , HIER=%STR(11 ,12 ));
     /*Neoplasm 4 */   %SET0(CC=11    , HIER=%STR(12 ));
     
From these files, the `logic` condition is located with regex, and then further split to determine the `condition` of the if statement, and the ccs that need to become `zeros` if the `condition` cc is `True`. For the above section of v12 this looks like:

                                          logic condition   zeros
    14                                     None       NaN     NaN
    15     cc5   =1 then do i=112;hcc(i)=0;end;         5     112
    16  cc7   =1 then do i=8,9,10;hcc(i)=0;end;         7  8,9,10
    17    cc8   =1 then do i=9,10;hcc(i)=0;end;         8    9,10
    18      cc9   =1 then do i=10;hcc(i)=0;end;         9      10
Lines that correspond to non-hierarchy rules are then dropped as either the `condition` or `zeros` is `NaN`. This DataFrame is then converted into a long format with one row for each Condition Code that needs to be zeroed. The resulting clean files are found in *ConditionCategory/* with the suffix *_rules*.

In [3]:
# Hierarchy rules for v12
df_rule = pd.read_csv('ConditionCategory/v12_rules.csv')
df_rule.head(7)

,to_zero,cc
0,112,5
1,8,7
2,9,7
3,10,7
4,9,8
5,10,8
6,10,9


## generate_hccs()
Given a list of recipients and diagnosis codes, returns a True/False table of HCCs for each unique recipient in the data. The input data should be formatted with 4 columns
* `recip_id` a unique identifier for a person.
* `icd` (string) an icd9 or icd10 diagnosis code. (No hanging whitespace)
* `version` (integer) {9 or 10} specifies whether a code is icd9 or icd10.
* `date` (datetime64) specifies the date of the diagnosis

In this example we will use fake data consiting of 127,504 diagnoses for 4999 unique people that occured in 2016. All diagnosis codes are icd10 and we will use version 21 of the CC mapping.

In [4]:
sample_data = pd.read_csv('sample_data.csv')
# Make sure date has the proper format, datetime64
sample_data['date'] = pd.to_datetime(sample_data.date)
sample_data.head()

,recip_id,icd,date,version
0,4224,Z7689,2016-01-01,10
1,4473,Z7689,2016-01-01,10
2,3348,G8929,2016-01-01,10
3,3956,Z7689,2016-01-01,10
4,1640,I776,2016-01-01,10


In [5]:
sample_data.dtypes

recip_id             int64
icd                 object
date        datetime64[ns]
version              int64
dtype: object

First, import all of the necessary cleaned tables from the above three functions. They will all be read into DataFrames. This is done by specifying a single string, `version`.

In [6]:
import os as os

# Specify the version of CCs to use.
version = 'v21'

# Determine the necessary crosswalks, hierarchy and CC list based on the version
crosswalk_list = [file for file in os.listdir('Crosswalks') if version in file]
hierarchy = 'ConditionCategory/'+version+'_rules.csv'
cc_list = 'ConditionCategory/'+version+'_labels.csv'

# Create icd to cc mapping DataFrame, and read in the rules and list DataFrames.
lst = []
for file in crosswalk_list:
    lst.append(pd.read_csv('Crosswalks/'+file))
    
df_map = pd.concat(lst)
df_hier = pd.read_csv(hierarchy)
df_list = pd.read_csv(cc_list)

`df_map` contains all v21 CC mappings for icd9 and icd10 from 2012 to 2017. 

In [7]:
df_map.head()

,cc,icd,version,year
0,2,0031,9,2012
1,115,00322,9,2012
2,39,00323,9,2012
3,39,00324,9,2012
4,115,0064,9,2012


`df_hier` contains all information on the hierarchal conditions to be applied at the end for v21. This DataFrame can be read as: for a given person if the condition category in `cc` is `True` set the condition category in `to_zero` to `False`.

In [8]:
df_hier.head()

,to_zero,cc
0,9,8
1,10,8
2,11,8
3,12,8
4,10,9


`df_list` contains all of the conditional categories and the labeling information. It is only useful for setting the index in the final table to ensure that all HCCs are represented.

In [9]:
df_list.head()

,cc,label
0,1,HIV/AIDS
1,2,"Septicemia, Sepsis, Systemic Inflammatory Resp..."
2,6,Opportunistic Infections
3,8,Metastatic Cancer and Acute Leukemia
4,9,Lung and Other Severe Cancers


With these tables, the Condition Categories are brought to the diagnosis table by merging with `df_map` on the icd code, the icd version, and the year.

In [10]:
merged = sample_data.merge(df_map, left_on=['icd', sample_data.date.dt.year.values, 'version'],
    right_on=['icd', 'year', 'version'], how='left')
merged.head(10)

,recip_id,icd,date,version,cc,year
0,4224,Z7689,2016-01-01,10,NaN,2016
1,4473,Z7689,2016-01-01,10,NaN,2016
2,3348,G8929,2016-01-01,10,NaN,2016
3,3956,Z7689,2016-01-01,10,NaN,2016
4,1640,I776,2016-01-01,10,108.0,2016
5,990,I776,2016-01-01,10,108.0,2016
6,3327,F802,2016-01-01,10,NaN,2016
7,4940,G8929,2016-01-01,10,NaN,2016
8,3563,Z7689,2016-01-01,10,NaN,2016
9,2616,Z7689,2016-01-01,10,NaN,2016


Many of the icd procedures do not map to any Condition Categories. After the merge, there is no need to keep track of the columns `date`, `version`, `year`, or `icd` so they are dropped in addition to all rows that were not assigned to a CC.

In [11]:
merged = merged.drop(columns=['date', 'version', 'year', 'icd'])
merged = merged[merged.cc.notnull()]
merged.head()

,recip_id,cc
4,1640,108.0
5,990,108.0
35,3003,108.0
36,3062,136.0
37,458,99.0


The number of rows and unique recipients left after dropping diagnoses that did not map are

In [12]:
print('Number of Observations:', len(merged))
print('Unique recipients left:', merged.recip_id.nunique())

Number of Observations: 14473
Unique recipients left: 4703


Next, this DataFrame of `recip_id` and `cc` is converted into a large truth table that counts whether a person has a Condition Category assigned to them.

In [13]:
merged = (merged.groupby(['recip_id', 'cc'])
            .size().unstack(fill_value=0)
            .reindex(df_list.cc, axis=1, fill_value=0).astype(bool))
merged[5:10]

cc,1,2,6,8,9,10,11,12,17,18,...,162,166,167,169,170,173,176,186,188,189
recip_id,,,,,,,,,,,,,,,,,,,,,
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
13,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


The majority of entries are `False`, so to check how many individuals have a `True` value for a given Category Code, look at the sum. 

In [14]:
merged.sum(axis=0)[0:10]

cc
1     113
2      67
6      11
8      35
9      35
10     31
11     34
12    103
17     52
18    550
dtype: int64

The final step is to apply the hierarchy conditions. For instance, from `df_hier` if CC8 is `True` CC11 must be set to `False`. There is one such an individual in the dataset who needs this hierarchy,

In [15]:
merged[(merged[8] == True) & (merged[11] == True)]

cc,1,2,6,8,9,10,11,12,17,18,...,162,166,167,169,170,173,176,186,188,189
recip_id,,,,,,,,,,,,,,,,,,,,,
561,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Also check that the hierarchy is only setting values to `False` when the `True`condition is met. The following recipients should not have their values for CC11 changed, as CC8, CC9 and CC10 are all `False`.

In [16]:
df_hier[df_hier.to_zero==11]

,to_zero,cc
2,11,8
5,11,9
7,11,10


In [17]:
cc11_remains = merged[(merged[8] == False) & (merged[9] == False) & 
       (merged[10] == False) & (merged[11] == True)].index
merged.loc[cc11_remains].head()

cc,1,2,6,8,9,10,11,12,17,18,...,162,166,167,169,170,173,176,186,188,189
recip_id,,,,,,,,,,,,,,,,,,,,,
101,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
306,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
408,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
771,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1113,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [18]:
# Apply heirarchies. For hierarchical codes, if the column in merged is True, 
# set the appropriate other column to False. 
for index, row in df_hier.iterrows():
    merged.loc[merged[row.cc] == True, row.to_zero] = False

The hierarchy was succesfully applied, and set CC11 to `False` because CC8 was `True`.

In [19]:
merged[merged.index==561]

cc,1,2,6,8,9,10,11,12,17,18,...,162,166,167,169,170,173,176,186,188,189
recip_id,,,,,,,,,,,,,,,,,,,,,
561,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


The people who should still have `True` in CC11 because neither CC8, CC9, nor CC10 were `True` still remain unchanged. Here I transposed the result for viewing purposes.

In [20]:
merged.loc[cc11_remains, 11].to_frame().T

recip_id,101,306,408,771,1113,1271,1382,1524,1630,1682,...,2938,2974,3498,3682,4000,4303,4316,4389,4576,4688
11,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


This process is written as a function in HCC.py so it can be performed with just `hcc_df = generate_hccs(sample_data, 'v21')`. Just make sure `sample_data` has the appropriate column names and corresponding datatypes.

In [21]:
import RiskAdjustment as ra
hcc_df = ra.generate_hccs(sample_data, 'v21')
hcc_df.head(10)

cc,1,2,6,8,9,10,11,12,17,18,...,162,166,167,169,170,173,176,186,188,189
recip_id,,,,,,,,,,,,,,,,,,,,,
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
